In [42]:
import pandas as pd
import seaborn as sns
pd.set_option('display.max_columns', None)
df = pd.read_csv("persona.csv")
df.head()

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [43]:
df.nunique()

PRICE       6
SOURCE      2
SEX         2
COUNTRY     6
AGE        46
dtype: int64

In [44]:
df.shape

(5000, 5)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [46]:
df["PRICE"].nunique()

6

In [47]:
df["PRICE"].value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [48]:
df.groupby("COUNTRY")["PRICE"].count()

COUNTRY
bra    1496
can     230
deu     455
fra     303
tur     451
usa    2065
Name: PRICE, dtype: int64

In [49]:
df.groupby("COUNTRY")["PRICE"].sum()

COUNTRY
bra    51354
can     7730
deu    15485
fra    10177
tur    15689
usa    70225
Name: PRICE, dtype: int64

In [50]:
df.groupby("SOURCE")["SOURCE"].count()

SOURCE
android    2974
ios        2026
Name: SOURCE, dtype: int64

In [51]:
df.groupby("COUNTRY").agg({"PRICE": ["mean"]})

,PRICE
,mean
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


In [52]:
df.groupby("SOURCE").agg({"PRICE": ["mean"]})

,PRICE
,mean
SOURCE,
android,34.174849
ios,34.069102


In [231]:
df.groupby(["COUNTRY","SOURCE"]).agg({"PRICE": ["mean"]})

PRICE
                      mean
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [240]:
# Görev 2 kısmı buradan sonrası
df_new = df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"PRICE": ["mean"]})
df_new.head()

PRICE
                                 mean
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897

In [260]:
# GÖREV 3: Çıktıyı PRICE'a göre sıralayınız.
agg_df = df_new.sort_values(by=("PRICE", "mean"), ascending=False)
agg_df

PRICE
                            mean
COUNTRY SOURCE  SEX    AGE      
bra     android male   46   59.0
usa     android male   36   59.0
fra     android female 24   59.0
usa     ios     male   32   54.0
deu     android female 36   49.0
...                          ...
usa     ios     female 38   19.0
                       30   19.0
can     android female 27   19.0
fra     android male   18   19.0
deu     android male   26    9.0

[348 rows x 1 columns]

In [261]:
# GÖREV 4: Indekste yer alan isimleri değişken ismine çeviriniz.
df_reset = agg_df.reset_index()
new_index = df_reset[("PRICE", "mean")]
df_reset.drop(("PRICE", "mean"), axis=1, inplace=True)
df_reset["PRICE"] = new_index
agg_df = df_reset
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE
,,,,,
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0
...,...,...,...,...,...
343,usa,ios,female,38,19.0
344,usa,ios,female,30,19.0
345,can,android,female,27,19.0


In [262]:
# GÖREV 5: AGE değişkenini kategorik değişkene çeviriniz ve agg_df'e ekleyiniz.
agg_df["AGE_CAT"] = df["AGE"].astype("category")
agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], 
                           bins=[0, 18, 23, 30, 40, 70], 
                           labels = ['0_18', '19_23', '24_30', '31_40', '41_70'])
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
,,,,,,
0,bra,android,male,46,59.0,41_70
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40
...,...,...,...,...,...,...
343,usa,ios,female,38,19.0,31_40
344,usa,ios,female,30,19.0,24_30
345,can,android,female,27,19.0,24_30


In [267]:
# GÖREV 6: Yeni level based müşterileri tanımlayınız ve veri setine değişken olarak ekleyiniz.
agg_df = agg_df.assign(customers_level_based = lambda x: x['COUNTRY'].str.upper() + '_' + x['SOURCE'].str.upper() + '_' + x['SEX'].str.upper() + '_' + x['AGE_CAT'].astype(str))
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based
,,,,,,,
0,bra,android,male,46,59.0,41_70,BRA_ANDROID_MALE_41_70
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40
...,...,...,...,...,...,...,...
343,usa,ios,female,38,19.0,31_40,USA_IOS_FEMALE_31_40
344,usa,ios,female,30,19.0,24_30,USA_IOS_FEMALE_24_30
345,can,android,female,27,19.0,24_30,CAN_ANDROID_FEMALE_24_30


In [269]:
agg_df.groupby("customers_level_based")["PRICE"].mean()

customers_level_based
BRA_ANDROID_FEMALE_0_18     35.645303
BRA_ANDROID_FEMALE_19_23    34.077340
BRA_ANDROID_FEMALE_24_30    33.863946
BRA_ANDROID_FEMALE_31_40    34.898326
BRA_ANDROID_FEMALE_41_70    36.737179
                              ...    
USA_IOS_MALE_0_18           33.983495
USA_IOS_MALE_19_23          34.901872
USA_IOS_MALE_24_30          34.838143
USA_IOS_MALE_31_40          36.206324
USA_IOS_MALE_41_70          35.750000
Name: PRICE, Length: 109, dtype: float64

In [270]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D","C","B","A"])
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based,SEGMENT
,,,,,,,,
0,bra,android,male,46,59.0,41_70,BRA_ANDROID_MALE_41_70,A
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40,A
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30,A
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40,A
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40,A
...,...,...,...,...,...,...,...,...
343,usa,ios,female,38,19.0,31_40,USA_IOS_FEMALE_31_40,D
344,usa,ios,female,30,19.0,24_30,USA_IOS_FEMALE_24_30,D
345,can,android,female,27,19.0,24_30,CAN_ANDROID_FEMALE_24_30,D


In [271]:
new_user = "TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based,SEGMENT
,,,,,,,,
18,tur,android,female,32,43.000000,31_40,TUR_ANDROID_FEMALE_31_40,A
35,tur,android,female,31,40.666667,31_40,TUR_ANDROID_FEMALE_31_40,A


In [274]:
new_user = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customers_level_based,SEGMENT
,,,,,,,,
208,fra,ios,female,40,33.000000,31_40,FRA_IOS_FEMALE_31_40,C
221,fra,ios,female,31,32.636364,31_40,FRA_IOS_FEMALE_31_40,C
